In [ ]:
using GuSTO
using AstrobeeRobot

In [ ]:
robot = Astrobee2D()
model = AstrobeeSE2()
env = Table();

In [ ]:
tbot2radius = 0.17
tbot2height = 0.41
tbot3radius = 0.05 # Update
tbot3height = 0.15

tbot2_pos = []
push!(tbot2_pos, [0.7, 0.7])
push!(tbot2_pos, [0.1, 0.])
push!(tbot2_pos, [0., -0.5])

tbot3_pos = []
push!(tbot3_pos, [0.5, -0.5])


for pos in tbot2_pos
    push!(env.obstacle_set, Cylinder(Point(pos..., -tbot2height+0.01), Point3(0., 0., tbot2height), tbot2radius))
end

In [ ]:
x_init = [-0.45;0.7;0;0;0;0]
x_goal = [0.7;-0.7;0;0;0;0];

In [ ]:
#### Code for showing Astrobee table workspace
vis = Visualizer()
delete!(vis)

vis[:goal]
setobject!(vis[:goal][:goal], 
    Object(HyperSphere(Point3(x_goal[1:2]..., -robot.r), 0.1),
        MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:table]
table_dim = env.worldAABBmax-env.worldAABBmin
setobject!(vis[:table][:table], 
        Object(HyperRectangle(env.worldAABBmin..., table_dim...),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

trans = Translation(x_init[1:2]..., -robot.r)
# rot = LinearMap(RotZ(0.)) ∘ LinearMap(RotY(0.))
rot = LinearMap(RotY(-pi/2))
settransform!(mvis["world"]["body"], trans ∘ rot)

trans = Translation(0., 1., -1.5)
rot = LinearMap(RotZ(-pi/2)) ∘ LinearMap(RotY(-0.5)) ∘ LinearMap(RotY(pi))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.7)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)

In [ ]:
function init_traj_table(TOP::TrajectoryOptimizationProblem{Astrobee2D{T}, AstrobeeSE2, E}) where {T,E}
  model, x_init, x_goal = TOP.PD.model, TOP.PD.x_init, TOP.PD.x_goal
  x_dim, u_dim, N, tf_guess = model.x_dim, model.u_dim, TOP.N, TOP.tf_guess
  N = TOP.N

  x_mid = [0.5; 0.5; 0.; zeros(3)]

  X = [hcat(linspace(x_init, x_mid, div(N,2))...) hcat(linspace(x_mid, x_goal, N-div(N,2))...)]
  U = zeros(u_dim, N)
  Trajectory(X, U, tf_guess)
end

In [ ]:
N = 36

x_init = [-0.25;0.4;0;0;0;0]
x_goal = [0.6;-0.5;0;0;0;0]
x_goal = [0.7;-0.7;pi/2;0;0;0]

x_init = [-0.25;0.4;0;0;0;0]
x_goal = [0.7;-0.5;0;0;0;0]

tf_guess = 41.

PD = ProblemDefinition(robot, model, env, x_init, x_goal)

TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOP)

SCPP = solve_SCP!(TOSgusto, TOP, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0);

In [ ]:
@show TOSgusto.SCPS.successful
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.iter_elapsed_times
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec
;

In [ ]:
accels, alphas = [], []
for k in 1:N-1
    push!(accels, 1/robot.mass*norm(TOSgusto.traj.U[1:2,k],2))
    push!(alphas, robot.Jinv[3,3]*norm(TOSgusto.traj.U[3,k],2))
end

PyPlot.figure()
PyPlot.plot(accels)
PyPlot.plot(robot.hard_limit_accel*ones(N-1),"k--")

PyPlot.figure()
PyPlot.plot(alphas)
PyPlot.plot(robot.hard_limit_alpha*ones(N-1),"k--")

In [ ]:
speeds, omegas = [], []
for k in 1:N
    push!(speeds, norm(TOSgusto.traj.X[4:5,k],2)) 
    push!(omegas, norm(TOSgusto.traj.X[6,k],2)) 
end
                            
PyPlot.figure()
PyPlot.plot(speeds)
PyPlot.plot(robot.hard_limit_vel*ones(N),"k--")

PyPlot.figure()
PyPlot.plot(omegas)
PyPlot.plot(robot.hard_limit_omega*ones(N),"k--")

In [ ]:
#### Code for animating Astrobee trajectories
vis = Visualizer()
delete!(vis)

vis[:goal]
setobject!(vis[:goal][:goal], 
    Object(HyperSphere(Point3(x_goal[1:2]..., robot.r), 0.1),
        MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:table]
table_dim = env.worldAABBmax-env.worldAABBmin
setobject!(vis[:table][:table], 
        Object(HyperRectangle(env.worldAABBmin..., table_dim...),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);
# q0 = 0.5*[1.; 1.; 1.; 1.]
q0 = sqrt(2)*[1.; 0.; -1.; 0.]
Qs = Vector{Float64}[]

trans = Translation(0., 1., -1.5)
rot = LinearMap(RotZ(-pi/2)) ∘ LinearMap(RotY(-0.5)) ∘ LinearMap(RotY(pi))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.7)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

speed_factor = 5

for k in 1:speed_factor:N
    q = [quat_inv(quat_multiply(mrp2quat([0.; 0.; tan(TOSgusto.SCPS.traj.X[3,k]/4)]), q0)); [TOSgusto.SCPS.traj.X[1:2,k]; -robot.r]]
    push!(Qs,q)
end

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)